In [2]:
!pip install arxiv


     ---------------------------------------- 81.3/81.3 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
  Running setup.py install for sgmllib3k: started
  Running setup.py install for sgmllib3k: finished with status 'done'


You should consider upgrading via the 'C:\Users\athar\Projects\Chatbot_openAI_Llama\Chatbot-OpenAI-Llama\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [86]:
from langchain_community.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [5]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
  Running setup.py install for wikipedia: started
  Running setup.py install for wikipedia: finished with status 'done'


You should consider upgrading via the 'C:\Users\athar\Projects\Chatbot_openAI_Llama\Chatbot-OpenAI-Llama\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [87]:
# Import Wikipedia API wrapper to fetch data from Wikipedia
api_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)

# This sets up a Wikipedia search tool that will return the top 1 result 
# and limit the content to 200 characters
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [88]:
# Now, if you use 'tool.run("your query")', it will fetch a short summary 
# of the most relevant Wikipedia page.
wiki.run("What is machine learning")

'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen'

In [90]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
vectordb=FAISS.from_documents(documents= documents,embedding= OpenAIEmbeddings())

retriever= vectordb.as_retriever()


In [91]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000247938F6670>, search_kwargs={})

In [92]:
# We use retrieval tool because, the agent will have freedom to choose or 
# not to choose the retriever. If needed it will be referred but if not needed, 
# other tools will be used.

from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever,"langsmith_search",
                      "Search for the information about langsmith. For any questions about Langsmith, you must use this tool")

In [93]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper= arxiv_wrapper)

In [94]:
arxiv.name

'arxiv'

In [95]:
tools=[wiki,arxiv,retriever_tool]

In [96]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\athar\\Projects\\Chatbot_openAI_Llama\\Chatbot-OpenAI-Llama\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for the information about langsmith. For any questions about Langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000247B6B34B80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIE

In [97]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OpenAI_API_KEY"]=os.getenv("OpenAI_API_KEY")

llm=ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
# from langchain import hub
# prompt=hub.pull("hwchase17/openai-functions-agent:51bac8c5")

# prompt.messages
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder

# Define the prompt template with the correct input variables
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")  # Ensure agent_scratchpad is included 
])
#We use MessagesPlaceholder because it will get updated as we use tools for the agents
# The 'agent_scratchpad' is used to store intermediate steps when the agent calls tools.
# - If the agent doesn't use tools, this can be removed.
# - If tools are used, LangChain requires 'agent_scratchpad' to process tool outputs before responding.
# - Keeping this empty at the start ensures the agent can use tools dynamically.

input_data = {
    "input": "what is langchain",
    "agent_scratchpad": ""  # Initial scratchpad value
}

In [116]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [117]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typin

In [118]:
agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChainLangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It serves as a language model integration framework.

> Finished chain.


{'input': 'what is langchain',
 'agent_scratchpad': [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\athar\\Projects\\Chatbot_openAI_Llama\\Chatbot-OpenAI-Llama\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
  ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
  Tool(name='langsmith_search', description='Search for the information about langsmith. For any questions about Langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000247B6B34B80>, 

In [119]:
print("Agent Scratchpad:", input_data["agent_scratchpad"])

Agent Scratchpad: [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\athar\\Projects\\Chatbot_openAI_Llama\\Chatbot-OpenAI-Llama\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)), ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)), Tool(name='langsmith_search', description='Search for the information about langsmith. For any questions about Langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000247B6B34B80>, retriever=VectorStoreRetriever(tags=['